In [ ]:
"""
Find the 2 users with the highest rank for their grade in each language
order by language name ascending, score descending, then grade id ascending
"""

In [ ]:
"""SQL table creation and data insertion"""
drop table if exists grade;
drop table if exists language;

CREATE TABLE `grade` (
`id` int(4) NOT NULL,
`language_id` int(4) NOT NULL,
`score` int(4) NOT NULL,
PRIMARY KEY (`id`));

CREATE TABLE `language` (
`id` int(4) NOT NULL,
`name` varchar(32) NOT NULL,
PRIMARY KEY (`id`));

INSERT INTO grade VALUES
(1,1,12000),
(2,1,13000),
(3,2,11000),
(4,2,10000),
(5,3,11000),
(6,1,11000),
(7,2,11000);


INSERT INTO language VALUES
(1,'C++'),
(2,'JAVA'),
(3,'Python');

In [ ]:
"""SQL solution"""
# 1
SELECT r.id, l.name, r.score
FROM
(SELECT a.id, a.language_id, a.score
FROM grade AS a, grade AS b
WHERE a.language_id = b.language_id
AND a.score <= b.score
GROUP BY a.language_id, a.id
HAVING COUNT(DISTINCT b.score) <= 2) AS r
INNER JOIN language AS l
ON r.language_id = l.id
ORDER BY l.name, r.score DESC, r.id

# 2
SELECT a.id, l.name, a.score
FROM
(SELECT id, language_id, score, DENSE_RANK() OVER (PARTITION BY language_id ORDER BY score DESC) AS r
FROM grade) AS a
INNER JOIN language AS l
ON a.language_id = l.id
WHERE r <= 2
ORDER BY l.name, a.score DESC, l.id

# 3
SELECT r.id, l.name, r.score
FROM
(SELECT id, language_id, score, DENSE_RANK() OVER(PARTITION BY language_id ORDER BY score DESC) AS sr
FROM grade) AS r, language AS l
WHERE r.language_id = l.id
AND r.sr <= 2
ORDER BY l.name, r.score DESC, r.id

# 4
SELECT a.id, l.name, a.score
FROM
(SELECT g.id, g.language_id, g.score,
IF(@pre_l = g.language_id, IF(@pre_s = g.score, @sr := @sr, @sr := @sr + 1), @sr := 1) AS srank,
 @pre_s := g.score, @pre_l := g.language_id
FROM grade AS g, (SELECT @sr := 0, @pre_s := -1, @pre_l := -1) AS r
ORDER BY g.language_id, g.score DESC) AS a
INNER JOIN language AS l
ON a.language_id = l.id
WHERE a.srank <= 2
ORDER BY l.name, a.score DESC, a.id

In [16]:
"""pandas dataframe creation"""
import pandas as pd

language = pd.DataFrame([[1, 'C++'],
                         [2, 'Java'],
                         [3, 'Python']],
                        columns=['id', 'name'])
grade = pd.DataFrame([(1,1,12000),
                      (2,1,13000),
                      (3,2,11000),
                      (4,2,10000),
                      (5,3,11000),
                      (6,1,11000),
                      (7,2,11000)],
                      columns=['id', 'language_id', 'score'])
grade.head()

,id,language_id,score
0,1,1,12000
1,2,1,13000
2,3,2,11000
3,4,2,10000
4,5,3,11000


In [5]:
language.head()

,id,name
0,1,C++
1,2,Java
2,3,Python


In [17]:
grade['rank'] = grade.groupby('language_id')['score'].rank(method='dense', ascending=False)
df = pd.merge(grade, language, left_on='language_id', right_on='id', how='inner', suffixes=['', '_language']).drop(columns='id_language')
df

,id,language_id,score,rank,name
0,1,1,12000,2.0,C++
1,2,1,13000,1.0,C++
2,6,1,11000,3.0,C++
3,3,2,11000,1.0,Java
4,4,2,10000,2.0,Java
5,7,2,11000,1.0,Java
6,5,3,11000,1.0,Python


In [18]:
df[df['rank'] <= 2].sort_values(by=['name','score','id'], ascending=[True, False, True])[['id', 'score', 'name']]

,id,score,name
1,2,13000,C++
0,1,12000,C++
3,3,11000,Java
5,7,11000,Java
4,4,10000,Java
6,5,11000,Python
